In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import popmachine, itertools, patsy, GPy, scipy
from popmachine.normal import MultivariateNormal

In [ ]:
machine = popmachine.Machine('sqlite:///../popmachine_local/.popmachine.db')

In [ ]:
ds = machine.search(include=['Strain'], **{'mM PQ':[0.0, 0.083, 0.333], 'M NaCl':['4.2', None]})
ds.meta.plate.unique()

In [ ]:
ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], include=['Strain'], **{'mM PQ':[0.0, 0.083, 0.333, 333], 'M NaCl':['4.2', None]})
ds.meta.Strain.unique()

In [ ]:
# ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
#        u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9'], Strain='ura3', **{'mM PQ':[0.0, 0.083, 0.333], 'M NaCl':['4.2', None]})

ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], Strain='ura3', **{'mM PQ':[0.0, 0.083, 0.33, 0.333], 'M NaCl':['4.2', None]})
ds.trim(10)
ds.log()
ds.poly_scale(2,groupby=['plate','mM PQ'])
ds.filter()

In [ ]:
plt.figure(figsize=(20,6))

ds.plot(columns=['plate'], colorby=['mM PQ'], buff=.3)


plt.savefig('figures/pq-ura3.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,9))

g = ds.meta.groupby(['plate', 'mM PQ'])

keys = ds.meta.plate.unique().tolist()
for k, ind in g:
    temp = ds.data.iloc[:,ind.index]
    
    p, pq = k
    
    i = keys.index(p)
    
    plt.subplot(3,3,i+1)
    
    
        
    if i == 0:
        if pq == '0.0':
            plt.plot(temp.index, temp.values[:,0], c='k', label='mM PQ = 0.0')
            plt.plot(temp.index, temp.values[:,1:], c='k')
        elif pq == '0.333':
            plt.plot(temp.index, temp.values[:,0], c='limegreen', label='mM PQ = 0.333')
            plt.plot(temp.index, temp.values[:,1:], c='limegreen')
        else:
            plt.plot(temp.index, temp.values[:,0], c='cyan', label='mM PQ = 0.083')
            plt.plot(temp.index, temp.values[:,1:], c='cyan')
        plt.legend()
        
    else:
        if pq == '0.0':
            plt.plot(temp.index, temp.values, c='k', label='mM PQ = 0.0')
        elif pq == '0.333':
            plt.plot(temp.index, temp.values, c='limegreen', label='mM PQ = 0.333')
        else:
            plt.plot(temp.index, temp.values, c='cyan', label='mM PQ = 0.083')
            
    if i % 3 == 0:
        plt.ylabel('log(OD)', fontsize=14)
    if i > 5:
        plt.xlabel('time (h)', fontsize=14)
            
    plt.ylim(ds.data.min().min(), ds.data.max().max())
    
plt.savefig('figures/ura3_PQ_data.pdf', bbox_inches='tight')

In [ ]:
pq0 = ds.meta['mM PQ'] == '0.0'
pq333 = ds.meta['mM PQ'] == '0.333'
pq083 = ds.meta['mM PQ'] == '0.083'

m = (ds.data.loc[:,pq083].mean(1) - ds.data.loc[:,pq0].mean(1))
s = np.sqrt(ds.data.loc[:,pq083].var(1) + ds.data.loc[:,pq0].var(1))

plt.plot(ds.data.index, m, c='cyan', label='mM PQ = 0.083')
plt.fill_between(ds.data.index, m-2*s, m+2*s, alpha=.4, color='cyan')

m = (ds.data.loc[:,pq333].mean(1) - ds.data.loc[:,pq0].mean(1))
s = np.sqrt(ds.data.loc[:,pq333].var(1) + ds.data.loc[:,pq0].var(1))

plt.plot(ds.data.index, m, c='limegreen', label='mM PQ = 0.333')
plt.fill_between(ds.data.index, m-2*s, m+2*s, alpha=.4, color='limegreen')

plt.plot([ds.data.index.min(), ds.data.index.max()], [0,0], c='k', lw=2)

plt.legend()

In [ ]:
pvals = []
tstats = []

for i in range(ds.data.shape[0]):
    
    p1 = ds.data.loc[ds.data.index[i],pq083]
    p2 = ds.data.loc[ds.data.index[i],pq0]
    
    ts, pv = scipy.stats.ttest_ind(p1, p2)
    
    pvals.append(pv)
    tstats.append(ts)
    
# plt.plot(ds.data.index, -np.log10(pvals), c='cyan', label='mM PQ = 0.083')
plt.plot(ds.data.index, tstats, c='cyan', label='mM PQ = 0.083')


pvals = []
tstats = []

for i in range(ds.data.shape[0]):
    
    p1 = ds.data.loc[ds.data.index[i],pq333]
    p2 = ds.data.loc[ds.data.index[i],pq0]
    
    ts, pv = scipy.stats.ttest_ind(p1, p2)
    
    pvals.append(pv)
    tstats.append(ts)
    
df = p1.shape[0] + p2.shape[0] - 2
t = scipy.stats.t(df=df)
    
# plt.plot(ds.data.index, -np.log10(pvals), c='limegreen', label='mM PQ = 0.333')
plt.plot(ds.data.index, tstats, c='limegreen', label='mM PQ = 0.333')


# plt.plot([ds.data.index.min(), ds.data.index.max()], [-np.log10(.05)]*2, c='k', lw=2)
plt.plot([ds.data.index.min(), ds.data.index.max()], [t.ppf(.05)]*2, c='k', lw=2)

plt.legend()

plt.ylabel('log(OD)', fontsize=14)
plt.xlabel('time (h)', fontsize=14)    
plt.savefig('figures/ura3_PQ_data_ttest.pdf', bbox_inches='tight')

In [ ]:
df = p1.shape[0] + p2.shape[0] - 2
t = scipy.stats.t(df=df)

In [ ]:
t.ppf(.05)

In [ ]:
t.cdf(tstats) - pvals

In [ ]:
ts, pv = scipy.stats.ttest_ind(p1, p2)

In [ ]:
ts, pv

# mM PQ = .083 

In [ ]:
# ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
#        u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9'], Strain='ura3', **{'mM PQ':[0.0, 0.083, 0.333], 'M NaCl':['4.2', None]})

ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], Strain='ura3', **{'mM PQ':[0.0, 0.083], 'M NaCl':['4.2', None]})
ds.trim(10)
ds.log()
ds.poly_scale(2,groupby=['plate','mM PQ'])
ds.filter()

In [ ]:
plt.figure(figsize=(20,6))

ds.plot(columns=['plate'], colorby=['mM PQ'], buff=.3)


plt.savefig('figures/pq-ura3-0.083.pdf', bbox_inches='tight')

In [ ]:
ds.data.shape

In [ ]:
# ds.data = ds.data.iloc[::8,:]

In [ ]:

melt = ds.melt()
# melt = melt.iloc[::10,:]
ind = np.random.choice(range(melt.shape[0]), 3000, replace=False)
melt = melt.iloc[ind,:]

plt.scatter(melt.time, melt.od, c=melt['mM PQ'])

melt.shape

In [ ]:
plt.figure(figsize=(20,6))
ds.plot(columns=['plate'], colorby=['mM PQ'], buff=.3)

In [ ]:
xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
xgp[['plate']], plate_labels = xgp.plate.factorize()
xgp[['plate-number']], _ = xgp['plate-number'].factorize()



del xgp['number']

xgp = xgp.values.astype(float)

ygp = melt['od'].values[:,None]

In [ ]:
import patsy

In [ ]:
xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
xgp['pq'] = xgp['mM PQ'].astype(float)
xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
xgp[['plate']], plate_labels = xgp.plate.factorize()
xgp[['plate-number']], _ = xgp['plate-number'].factorize()

xgp = patsy.dmatrix('scale(time) + scale(pq) + plate + number + 0', xgp)
designInfo = xgp.design_info

In [ ]:
xgp = np.array(xgp)

In [ ]:
kmain = GPy.kern.RBF(2, ARD=True, name='main')
kbatch = GPy.kern.RBF(2, ARD=True, name='batch')
krep = GPy.kern.RBF(2, ARD=True, name='rep')
# kern = GPy.kern.Hierarchical((kmain, kbatch))
kern = GPy.kern.Hierarchical((kmain, kbatch, krep))

kern

In [ ]:
m = GPy.models.GPRegression(xgp, ygp, kern, normalizer=True)
m.randomize()
m

In [ ]:
m.optimize()
m

In [ ]:
m['.*lengthscale']

In [ ]:
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = xgp[:,1].min()
xpred[50:,1] = xgp[:,1].max()
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
pred = popmachine.normal.MultivariateNormal(mu, cov)
pred.plot()

In [ ]:
op = np.zeros((50,100))
op[:,:50] = -np.eye(50)
op[:,50:] = np.eye(50)
pred.dot(op).plot()

In [ ]:
plt.plot(scipy.stats.multivariate_normal.rvs(pred.dot(op).mean, pred.dot(op).cov, size=10).T);

In [ ]:
# xpred[:,2] = 1000
# xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True)
pred = popmachine.normal.MultivariateNormal(mu, cov)
# pred.dot(op).plot(xpred[:50,0])
pred.plot()

# mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
# pred = popmachine.normal.MultivariateNormal(mu, cov)
# pred.dot(op).plot(xpred[:50,0],color='red')
    
plt.tight_layout()

In [ ]:
# xpred[:,2] = plate_labels.tolist().index(u'20161107_PQ_osmo_combo')

# mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=GPy.kern.Hierarchical([m.kern.main, m.kern.batch]))
# tempPred = popmachine.normal.MultivariateNormal(mu, cov, )

# # pred.dot(op).plot()
# # tempPred.dot(op).plot(color='r')

# pred.plot()
# tempPred.plot(color='r')

In [ ]:
m.kern = GPy.kern.Hierarchical([kmain, kbatch, krep])

In [ ]:
op = np.zeros((50,100))
op[:,:50] = -np.eye(50)
op[:,50:] = np.eye(50)
tempPred.dot(op).plot()

In [ ]:
mnull = GPy.models.GPRegression(xgp, ygp, GPy.kern.RBF(2, ARD=True), normalizer=True)
mnull.randomize()
mnull.optimize()
mnull

In [ ]:
m.kern

In [ ]:
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = xgp[:,1].min()
xpred[50:,1] = xgp[:,1].max()
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
pred = popmachine.normal.MultivariateNormal(mu, cov)

nullPred = popmachine.normal.MultivariateNormal(*mnull.predict_noiseless(xpred, full_cov=True))

pred.dot(op).plot()
nullPred.dot(op).plot(color='r')

In [ ]:
models = {}
for p in [u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4']:

    ds = machine.search(plates=[p], Strain='ura3', **{'mM PQ':[0.0, 0.083], 'M NaCl':['4.2', None]})
    ds.trim(10)
    ds.log()
    ds.poly_scale(2,groupby=['plate','mM PQ'])
    ds.filter()
    #ds.data = ds.data.iloc[::8,:]
    
    melt = ds.melt()
    melt = melt.iloc[::10,:]
    
    xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
    #xgp[['plate']] = xgp.plate.factorize()[0]
    xgp['pq'] = xgp['mM PQ'].astype(float)
    xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
    xgp[['plate']], _ = xgp.plate.factorize()
    xgp[['plate-number']], _ = xgp['plate-number'].factorize()
    
    xgp = patsy.build_design_matrices([designInfo], xgp)[0]
    xgp = np.array(xgp)
    xgp = xgp.astype(float)

    ygp = melt['od'].values[:,None]

    kmain = GPy.kern.RBF(2, ARD=True, name='main')

    temp = GPy.models.GPRegression(xgp, ygp, kmain)
    temp.randomize()
    temp.optimize()
    
    print p, temp.log_likelihood()
    
    models[p] = temp
    

In [ ]:
plt.figure(figsize=(9,9))

for i, p in enumerate(models.keys()):    
    
    plt.subplot(3, 3, i+1)
    plt.title(p)
    
    temp = models[p]
    
    mu, cov = temp.predict_noiseless(xpred, full_cov=True)
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    #pred.dot(op).plot(xpred[:50,0])
    tempPred.dot(op).plot(xpred[:50,0],color='r')
    
    xpred[:,2] = plate_labels.tolist().index(p)
    xpred[:,3] = 1000
    mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=GPy.kern.Hierarchical([m.kern.main.copy(), m.kern.batch.copy()]))    
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    #pred.dot(op).plot(xpred[:50,0])
    tempPred.dot(op).plot(xpred[:50,0],color='g')
    
    ds = machine.search(plates=[p], Strain='ura3', **{'mM PQ':[0.0, 0.083], 'M NaCl':['4.2', None]})
    ds.trim(10)
    ds.log()
    ds.poly_scale(2,groupby=['mM PQ'])
    ds.filter()
    
    ds.data = ds.data.loc[ds.data.index>xpred[:,0].min(), :]
    ds.data = ds.data.loc[ds.data.index<xpred[:,0].max(), :]
    
    select = np.isclose(0, ds.meta['mM PQ'].values.astype(float))
    diff = ds.data.loc[:,~select].mean(1) - ds.data.loc[:,select].mean(1)
    
    plt.plot(diff.index.values, diff.values, c='k', lw=2)
    
    plt.ylim(-.65,.3)
    
    if i > 5:
        plt.xlabel('time (normalized)', fontsize=12)
        
    if i % 3 == 0:
        plt.ylabel('OD (normalized)', fontsize=12)
    
plt.tight_layout()

In [ ]:
tempPred.dot(op).plot(xpred[:50,0],color='g')
pred.dot(op).plot(xpred[:50,0],color='b')

In [ ]:
plt.figure(figsize=(9,9))

for i, p in enumerate(models.keys()):    
    plt.title(p)
    
    temp = models[p]
    
    mu, cov = temp.predict_noiseless(xpred, full_cov=True)
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    
    tempPred.dot(op).plot(xpred[:50,0],color='r')
       
    plt.ylim(-.65,.3)
    
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[50:,1] = 0.083
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True)
pred = popmachine.normal.MultivariateNormal(mu, cov)
pred.dot(op).plot(xpred[:50,0])
    
plt.tight_layout()

In [ ]:
np.isclose(0, ds.meta['mM PQ'].values.astype(float))

In [ ]:
melt = ds.melt()
melt.head()

In [ ]:
xgp = melt[['time', 'mM PQ', 'plate']].copy()
xgp[['plate']] = xgp.plate.factorize()[0]
xgp = xgp.values.astype(float)

ygp = melt['od'].values[:,None]

In [ ]:
kmain = GPy.kern.RBF(2, ARD=True, name='main')
kbatch = GPy.kern.RBF(2, ARD=True, name='batch')
kern = GPy.kern.Hierarchical((kmain, kbatch))

kern

In [ ]:
xgp[::100,:]

In [ ]:
m = GPy.models.GPRegressionGrid(xgp[::100,:2], ygp[::100,:], GPy.kern.RBF(input_dim=2, variance=1, ARD=True))

In [ ]:
g = melt[['time', 'mM PQ', 'od']].groupby(['time', 'mM PQ'])
app = g.apply(mean)
app = app['od']

app

In [ ]:
import pandas as pd

In [ ]:
ygp = app.reshape((87, 3))
x0 = app.index.get_level_values(0).values
x1 = app.index.get_level_values(1).values.astype(float)[:3]

In [ ]:
x1

In [ ]:
m = GPy.models.GPKroneckerGaussianRegression(x0, x1, ygp, GPy.kern.RBF(), GPy.kern.RBF())

In [ ]:
ygp = app.values.reshape((87, 3))
xgp = np.array(app.index.tolist()).astype(float)
ygp = ygp.flatten()[:,None]
m = GPy.models.GPRegressionGrid(xgp, ygp, GPy.kern.RBF(input_dim=2, variance=1, ARD=True))
m.randomize()
m

In [ ]:
m.optimize()
m

In [ ]:
m.kern.lengthscale

In [ ]:
xpred = np.zeros((50, 2))
xpred[:,0] = np.linspace(ds.data.index.min(), ds.data.index.max())

mu, cov = m.predict(xpred, full_cov=False)
pred = popmachine.normal.MultivariateNormal(mu, np.eye(50)*cov)
pred.plot()

In [ ]:
(48-4.5)*2

In [ ]:
GPy.models.GPKroneckerGaussianRegression()

In [ ]:
4*87

# mM PQ = .333 

In [ ]:
# ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
#        u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9'], Strain='ura3', **{'mM PQ':[0.0, 0.083, 0.333], 'M NaCl':['4.2', None]})

ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], Strain='ura3', **{'mM PQ':[0.0, 0.333, 333.0], 'M NaCl':['4.2', None]})
ds.trim(10)
ds.log()
ds.poly_scale(2,groupby=['plate','mM PQ'])
ds.filter()

In [ ]:
ds.meta.loc[ds.meta['mM PQ'] == '333.0', 'mM PQ'] = '0.333'

In [ ]:
plt.figure(figsize=(20,6))

ds.plot(columns=['plate'], colorby=['mM PQ'], buff=.3)


plt.savefig('figures/pq-ura3-0.333.pdf', bbox_inches='tight')

In [ ]:
melt = ds.melt()
# melt = melt.iloc[::10,:]
ind = np.random.choice(range(melt.shape[0]), 3000, replace=False)
melt = melt.iloc[ind,:]

plt.scatter(melt.time, melt.od, c=melt['mM PQ'])

melt.shape

In [ ]:
xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
xgp['pq'] = xgp['mM PQ'].astype(float)
xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
xgp[['plate']], plate_labels = xgp.plate.factorize()
xgp[['plate-number']], _ = xgp['plate-number'].factorize()

xgp = patsy.dmatrix('scale(time) + scale(pq) + plate + number + 0', xgp)
designInfo = xgp.design_info

In [ ]:
xgp = np.array(xgp)

In [ ]:
kmain = GPy.kern.RBF(2, ARD=True, name='main')
kbatch = GPy.kern.RBF(2, ARD=True, name='batch')
krep = GPy.kern.RBF(2, ARD=True, name='rep')
# kern = GPy.kern.Hierarchical((kmain, kbatch))
kern = GPy.kern.Hierarchical((kmain, kbatch, krep))

kern

In [ ]:
m = GPy.models.GPRegression(xgp, ygp, kern, normalizer=True)
m.randomize()
m

In [ ]:
m.optimize()
m

In [ ]:
m['.*lengthscale']

In [ ]:
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = xgp[:,1].min()
xpred[50:,1] = xgp[:,1].max()
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
pred = popmachine.normal.MultivariateNormal(mu, cov)
pred.plot()

In [ ]:
op = np.zeros((50,100))
op[:,:50] = -np.eye(50)
op[:,50:] = np.eye(50)
pred.dot(op).plot()

## other strains 

In [ ]:
ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], 
                    Strain=['ura3', 'rosR', 'hrg', 'VNG0194H', 'hlx2', 'hlx1', 'sirR', 'prp2', 'cspD2', 'arcR'], 
                    **{'mM PQ':[0.0, 0.083, 0.333, 333.0], 'M NaCl':['4.2', None]})
ds.trim(10)
ds.log()
ds.poly_scale(2,groupby=['plate','mM PQ'])
ds.filter()

In [ ]:
plt.figure(figsize=(18,12))

ds.plot(columns=['plate'], rows=['Strain'], colorby=['mM PQ'], buff=.3)
plt.tight_layout()

plt.savefig('figures/pq-strain.pdf', bbox_inches='tight')